In [7]:
import requests
import json

In [8]:
text = """
Changing the IAM policy for an AWS App Runner service using the AWS CLI typically means updating the service to use a **different IAM role**.

There are two primary IAM roles associated with an App Runner service that you might want to change:

1.  **Access Role:** This role grants App Runner permission to pull your source code or container image from a repository (like Amazon ECR).
2.  **Instance Role:** This role is assumed by your running application container, giving it permissions to access other AWS services (like S3, DynamoDB, or SQS).

The process involves updating your App Runner service configuration to point to a new IAM role's ARN.

Here is a complete step-by-step guide.

### Before You Begin: Create the New IAM Role

You can't change a policy on the fly; you must first create a new IAM Role with the desired policies attached and then tell App Runner to use that new role.

Let's assume you want to create a new role with updated permissions.

1.  **Create the IAM Policy (JSON file):**
    Create a file, e.g., `my-new-app-permissions.json`, with the permissions your application needs.
    ```json
    // my-new-app-permissions.json
    {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "s3:GetObject",
                    "s3:PutObject"
                ],
                "Resource": "arn:aws:s3:::my-app-bucket/*"
            },
            {
                "Effect": "Allow",
                "Action": "dynamodb:Query",
                "Resource": "arn:aws:dynamodb:us-east-1:123456789012:table/my-app-table"
            }
        ]
    }
    ```

2.  **Create the Trust Policy (JSON file):**
    This policy allows the App Runner service to assume the role.
    *   For an **Instance Role** (your running app), use `tasks.apprunner.amazonaws.com`.
    *   For an **Access Role** (pulling from ECR), use `build.apprunner.amazonaws.com`.

    Create a file, e.g., `apprunner-trust-policy.json`:
    ```json
    // apprunner-trust-policy.json
    {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "tasks.apprunner.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    ```

3.  **Use the AWS CLI to Create the Role and Attach the Policy:**
    Replace `MyNewAppRunnerInstanceRole` with your desired role name.

    ```bash
    # Create the IAM role
    aws iam create-role \
        --role-name MyNewAppRunnerInstanceRole \
        --assume-role-policy-document file://apprunner-trust-policy.json

    # Create the IAM policy from the JSON file
    aws iam create-policy \
        --policy-name MyNewAppRunnerPermissionsPolicy \
        --policy-document file://my-new-app-permissions.json

    # Attach the policy to the role (replace with your AWS Account ID)
    aws iam attach-role-policy \
        --role-name MyNewAppRunnerInstanceRole \
        --policy-arn arn:aws:iam::123456789012:policy/MyNewAppRunnerPermissionsPolicy
    ```

Now you have a new IAM role ready to be used. Note its ARN, which will look like: `arn:aws:iam::123456789012:role/MyNewAppRunnerInstanceRole`.

---

### Step 1: Get Your App Runner Service ARN

You need the unique identifier (ARN) for the service you want to update.

```bash
aws apprunner list-services
```

This will return a list of your services. Find the `ServiceArn` for the one you want to modify.

### Step 2: Update the App Runner Service with the New IAM Role

The command to use is `aws apprunner update-service`. The specific parameter depends on which role you are changing.

#### Example 1: Changing the Instance Role (for your running application)

This is the most common scenario. You are giving your application code new or different permissions to other AWS services.

Use the `--instance-configuration` parameter.

```bash
# Replace the placeholders with your actual values
SERVICE_ARN="arn:aws:apprunner:us-east-1:123456789012:service/my-app/a1b2c3d4e5"
NEW_INSTANCE_ROLE_ARN="arn:aws:iam::123456789012:role/MyNewAppRunnerInstanceRole"

aws apprunner update-service \
    --service-arn "$SERVICE_ARN" \
    --instance-configuration "InstanceRoleArn=$NEW_INSTANCE_ROLE_ARN"
```

This command initiates a new deployment of your service with the updated instance role.

#### Example 2: Changing the Access Role (for pulling from ECR)

You would do this if you moved your ECR repository or need to adjust the permissions App Runner uses to access ECR.

Use the `--source-configuration` parameter.

```bash
# First, create an ECR access role if you don't have one
# The trust policy should use "build.apprunner.amazonaws.com"
# The permissions policy needs ECR pull rights (e.g., AWSAppRunnerServicePolicyForECRAccess)

# Replace the placeholders with your actual values
SERVICE_ARN="arn:aws:apprunner:us-east-1:123456789012:service/my-app/a1b2c3d4e5"
NEW_ACCESS_ROLE_ARN="arn:aws:iam::123456789012:role/MyNewECRAccessRole"

aws apprunner update-service \
    --service-arn "$SERVICE_ARN" \
    --source-configuration "AuthenticationConfiguration={AccessRoleArn=$NEW_ACCESS_ROLE_ARN}"
```

**Note:** When you update a complex parameter like `source-configuration`, you often need to provide the full structure. If you only provide the `AuthenticationConfiguration`, other settings within `source-configuration` (like `ImageRepository`) might be reset. It's safer to get the current configuration first. See the "Important Considerations" section.

### Step 3: Verify the Change

After running the `update-service` command, you can confirm the change was applied.

1.  **Check the deployment status:** The update triggers a new deployment. You can check its status in the App Runner console or via the CLI:
    ```bash
    aws apprunner list-operations --service-arn "$SERVICE_ARN"
    ```
    Look for an operation of type `UPDATE_SERVICE` and check its `Status`. It should progress from `IN_PROGRESS` to `SUCCEEDED`.

2.  **Describe the service:** Once the deployment is complete, describe the service to see the new configuration.

    ```bash
    aws apprunner describe-service --service-arn "$SERVICE_ARN"
    ```
    In the JSON output, check the value of:
    *   `Service.InstanceConfiguration.InstanceRoleArn` (for the instance role)
    *   `Service.SourceConfiguration.AuthenticationConfiguration.AccessRoleArn` (for the access role)

### Important Considerations

*   **Updating only one part of a configuration:** When using `update-service`, you only need to specify the parameters you are changing. For example, if you are only changing the instance role, you do not need to respecify the `source-configuration`. The service will retain its existing source configuration.
*   **Permissions:** The user or role executing the `aws apprunner update-service` command needs the `apprunner:UpdateService` permission. They also need the `iam:PassRole` permission for the new IAM role you are assigning to the service. This is a security measure to prevent privilege escalation.
"""

In [9]:
url = "https://96ef-202-54-141-35.ngrok-free.app/api/kb/"

payload = json.dumps({"text": text})
headers = {"Content-Type": "application/json"}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


{"message":"Document inserted successfully"}
